<h1 style="background-color:#4E9A06; color:#ffffff; padding:10px 15px; border-radius:5px; margin-top:1rem; margin-bottom:1rem; text-align:center;">
  02b – Interactive EDA for stakeholders
</h1>

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Objectives
</h2>
<ul>
  <li><strong>Provide non-technical stakeholders</strong> with intuitive, interactive visualizations of key findings from our climate-disease dataset.</li>
  <li><strong>Highlight</strong> spatial, temporal, and anomaly-driven disease risks in wild vs. agricultural systems.</li>
  <li><strong>Enable exploration</strong> of model insights (e.g. feature importances) without code.</li>
</ul>

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Inputs
</h2>
<ul>
  <li><strong>Cleaned dataset</strong>: <code>data/processed/merged_climate_disease_final.csv</code> (ETL-processed, anomalies computed, incidence zones assigned).</li>
  <li><strong>Model outputs</strong>: Pre-computed feature importances and OLS interaction summaries for Hypotheses 1–5.</li>
  <li><strong>Geographic metadata</strong>: Latitude/longitude, system type (Ag vs. Wild), pathogen group.</li>
</ul>

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Outputs
</h2>
<ul>
  <li><strong>Interactive world map</strong> showing survey locations colored by temperature and shaped by system type.</li>
  <li><strong>Anomaly vs. incidence plots</strong> (temperature &amp; precipitation) with LOWESS smoothing, colored by system, pathogen group, and transmission mode.</li>
  <li><strong>Stacked bar charts</strong> of pathogen distributions and host orders.</li>
  <li><strong>Feature importance dashboard</strong> (bar chart or table) to reveal top predictors from our ML models.</li>
</ul>

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Additional Comments
</h2>
<ul>
  <li>All visuals are implemented with Plotly/Streamlit for seamless interactivity.</li>
  <li>Designed for non-technical audiences: tooltips, legends, and clear titles.</li>
  <li>Future enhancements: geographic filtering sliders, time-series animations, exportable snapshots.</li>
</ul>
<hr>


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  2.0 Environment Setup & Imports
</h2>

In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import numpy as np
import os

# Add style_loader path manually
STYLE_DIR = Path.cwd() / "notebook_style"
if str(STYLE_DIR) not in sys.path:
    sys.path.insert(0, str(STYLE_DIR))

from style_loader import load_style

load_style()
print("Style loaded successfully")

# Fallback safety
WILD_COLOR = globals().get("WILD_COLOR", "#66c2a5")
AG_COLOR = globals().get("AG_COLOR", "#3288bd")


Injected custom CSS from: c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\jupyter_notebooks\notebook_style\custom.css
Style loaded successfully


<!-- ──────────────────────────────────────────────────────────────────────── -->
<!-- Cell 2: Section Header (H2) “2.1 Imports & Load Data”                     -->
<!-- ──────────────────────────────────────────────────────────────────────── -->
<h2 style="color:#4E9A06; margin-top:1rem; margin-bottom:0.5rem;">
  2.1 Load Data
</h2>


In [19]:
# Load cleaned dataset from previous step
PROCESSED_DIR = Path("../data/processed")
cleaned_fp = PROCESSED_DIR / "merged_climate_disease_final.csv"

df = pd.read_csv(cleaned_fp)
print("Shape:", df.shape)
df.head()

Shape: (4932, 56)


,study_id,First_author,Study_Year,Person_extracting,Date_extracted,Paper_double_checked,study_reference,Host.species,Host_family,Host_order,...,start_date,monthly_temp,contemp_temp,contemp_precip,.geo,calculated_incidence,duration_mo,temp_anomaly,rain_anomaly,incidence_zone
0,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2008-08-01,181.284576,288.956314,0.005280,"{""type"":""Polygon"",""coordinates"":[[[6.028999805...",0.000000,1,107.671738,-78.742580,Low
1,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2008-08-01,133.753340,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.117000102...",0.000000,1,154.745943,-96.430852,Low
2,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2008-08-01,129.520853,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.122000217...",0.000000,1,158.978430,-98.153206,Low
3,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2008-08-01,112.943493,288.128423,0.005225,"{""type"":""Polygon"",""coordinates"":[[[6.147999763...",0.027778,1,175.184930,-106.300282,Low
4,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2008-08-01,189.015197,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.083000183...",0.000000,1,99.484086,-77.163348,Low


In [ ]:
df.columns.tolist()

In [47]:
# Recompute the needed columns:

# (a) Convert the survey (contemporaneous) temperature from K → °C
df["contemp_temp_C"] = df["contemp_temp"] - 273.15

# (b) Convert WorldClim “monthly_temp” (tenths of °C) → °C
df["monthly_temp_C"] = df["monthly_temp"] / 10.0

# (c) Convert WorldClim “annual_mean_temp” (tenths of °C) → °C
df["annual_mean_temp_C"] = df["annual_mean_temp"] / 10.0

# ────────────────────────────────────────────────────────────────────────────────
# 2) Compute the true Celsius‐based temperature anomaly:
# ────────────────────────────────────────────────────────────────────────────────

df["temp_anomaly_C"] = df["contemp_temp_C"] - df["monthly_temp_C"]

# ────────────────────────────────────────────────────────────────────────────────
# 3) Re‐compute rainfall anomalies (if not already correct):
#
#   - contemp_precip is already mm/day
#   - monthly_precip and annual_precip are in mm/month → convert to mm/day by dividing by 30
# ────────────────────────────────────────────────────────────────────────────────

# 1) Convert everything to mm/day
df["contemp_precip_mm_day"] = df["contemp_precip"] * 1000  # m d⁻¹ → mm d⁻¹
df["monthly_precip_mm_day"] = df["monthly_precip"] / 30.0  # mm month⁻¹ → mm d⁻¹
df["annual_precip_mm_day"] = df["annual_precip"] / 365.0  # mm yr⁻¹   → mm d⁻¹

# 2) Compute the anomaly in mm/day
df["rain_anomaly_mm_day"] = df["contemp_precip_mm_day"] - df["monthly_precip_mm_day"]

# 2) Absolute temperature anomaly
df["abs_temp_anom"] = (df["monthly_temp_C"] - df["contemp_temp_C"]).abs()

# 3) Absolute precipitation anomaly
#    raw monthly_precip is in mm/day already
df["abs_precip_anom"] = (df["monthly_precip_mm_per_day"] - df["contemp_precip"]).abs()

# 4) Standardize latitude/longitude column names
#    (ensure capitals if your plotting code expects 'Latitude','Longitude')
df.rename(columns={"latitude": "Latitude", "longitude": "Longitude"}, inplace=True)

# Verify
needed = ["contemp_temp_C", "abs_temp_anom", "abs_precip_anom", "Latitude", "Longitude"]
print("New columns present:", all(col in df.columns for col in needed))
df.head()

New columns present: True


,study_id,First_author,Study_Year,Person_extracting,Date_extracted,Paper_double_checked,study_reference,Host.species,Host_family,Host_order,...,monthly_precip_mm_per_day,rain_anomaly_daily,annual_precip_mm_per_day,abs_temp_anom,abs_precip_anom,System,contemp_precip_mm_day,monthly_precip_mm_day,annual_precip_mm_day,rain_anomaly_mm_day
0,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2.624929,-2.619648,31.449791,2.322144,2.619648,Wild,5.280471,2.624929,2.584914,2.655542
1,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,3.214538,-3.209250,42.772176,1.973949,3.209250,Wild,5.288383,3.214538,3.515521,2.073845
2,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,3.271950,-3.266661,43.910861,2.397197,3.266661,Wild,5.288383,3.271950,3.609112,2.016433
3,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,3.543517,-3.538292,48.609279,3.684074,3.538292,Wild,5.224704,3.543517,3.995283,1.681187
4,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,2.572288,-2.567000,29.943766,3.552237,2.567000,Wild,5.288383,2.572288,2.461131,2.716095


We will plot visualizations that might be interesting for our stakeholders:

- Technical teams
- Farmers
- Environmental agency

## 1. Mapping the Landscape (Global Map of Survey Sites)
Goal: Display the geographic spread of surveys, with markers shaped by system type (Agricultural vs. Wild) and colored by average temperature.

In [48]:
import plotly.graph_objects as go
import plotly.io as pio

# 1) Make sure you have the System column
df["System"] = df["system_type"].map({"Ag": "Agricultural", "Natural": "Wild"})


fig = go.Figure()

# Add one trace per system
for system, symbol in [("Agricultural", "circle"), ("Wild", "diamond")]:
    df_sub = df[df["System"] == system]
    fig.add_trace(
        go.Scattergeo(
            lon=df_sub["Longitude"],
            lat=df_sub["Latitude"],
            mode="markers",
            name=system,
            marker=dict(
                symbol=symbol,
                size=8,
                line=dict(width=0.5, color="black"),
                color=df_sub["contemp_temp_C"],
                coloraxis="coloraxis",
                colorscale="Inferno",
                colorbar=dict(
                    title="Temp (°C)",
                    tickformat=".0f",
                    x=-0.12,  # left of map
                    y=0.5,
                    yanchor="middle",
                    len=0.6,
                    thickness=15,
                    outlinecolor="black",
                ),
            ),
            hovertemplate=(
                "<b>%{text}</b><br><br>"
                "Temp: %{marker.color:.1f} °C<br>"
                "Incidence: %{customdata[0]:.2f}<br>"
                "Year: %{customdata[1]}<br>"
                "Agent: %{customdata[2]}<br>"
                "Host order: %{customdata[3]}<extra></extra>"
            ),
            text=df_sub["System"],
            customdata=df_sub[
                ["calculated_incidence", "Study_Year", "Parasite_or_pest", "Host_order"]
            ].values,
        )
    )

# 2) Configure the shared coloraxis (single bar)
fig.update_layout(
    coloraxis=dict(
        colorscale="Inferno",
        colorbar=dict(
            title="Temp (°C)",
            tickformat=".0f",
            x=0.02,  # move colorbar to left margin
            y=0.5,
            yanchor="middle",
            len=0.6,
            thickness=15,
            outlinecolor="black",
        ),
    )
)
# 3) Update layout & projection
fig.update_geos(
    visible=False,
    showcountries=True,
    countrycolor="lightgray",
    showland=True,
    landcolor="whitesmoke",
)
fig.update_layout(
    legend=dict(
        title="System",
        x=0.02,  # far left
        y=0.98,  # top
        xanchor="left",
        yanchor="top",
        bgcolor="rgba(0,0,0,0)",
        font=dict(size=12),
    ),
    title=(
        "Global database of plant disease spanning geography, weather and climate<br>"
        "Circles = Agricultural, Diamonds = Wild (N=4339). color = mean temp (°C)."
    ),
    title_x=0.5,
    title_font_size=14,
    margin=dict(l=180, r=20, t=100, b=20),
    width=1000,
    height=600,
    template="plotly",
)


# 4) Show inline (Notebook mode)

pio.renderers.default = "notebook_connected"
fig.show()

# 3) Export to high-res PNG at ~300 dpi
#    width_inches = width_pixels / dpi → here: 1000px / 300dpi ≈ 3.33"
#    so scale=3 gives 3000px → 300dpi at ~10" wide
fig.write_image(
    "../images/global_map.png", format="png", width=1000, height=600, scale=3
)
print("Saved high-res PNG to images/global_map.png")

Saved high-res PNG to images/global_map.png


Technical teams:

- Shows strong clustering in mid‐latitudes and Africa—consider potential spatial sampling bias.

- Color gradient confirms wide temperature coverage (10–30 °C), ideal for model generalization.

Farmer:

- See our region’s marker density—are we under‐ or over‐represented?

- Hover to compare local incidence rates and survey years.

Environmental Agency:

- Identify “cold‐spots” (few surveys) for targeted wild‐plant disease monitoring.

- Map guides resource allocation for new survey campaigns.

## 2.Climate Variable Correlations (Correlation Heatmap)
Goal: Show how different climate variables correlate with each other. Dark blue indicates strong positive correlation, while dark red shows strong negative correlation.

In [74]:
import plotly.express as px

# Compute correlation matrix
corr_matrix = df[
    [
        "annual_mean_temp",
        "monthly_temp",
        "contemp_temp",
        "temp_anomaly",
        "annual_precip_mm_per_day",
        "monthly_precip_mm_per_day",
        "contemp_precip_mm_day",
        "rain_anomaly_daily",
    ]
].corr()

# 2) Plot with a blue-white-red diverging scale
fig = px.imshow(
    corr_matrix,
    labels=dict(x="Climate Metric", y="Climate Metric", color="Pearson r"),
    x=climate_cols,
    y=climate_cols,
    color_continuous_scale="RdBu",  # reversed Red-Blue: blue=+1, red=-1
    zmin=-1,
    zmax=1,
    text_auto=".2f",
    aspect="auto",
)
fig.update_layout(
    title="Correlation Matrix of Climate Variables",
    width=700,
    height=700,
    template="plotly_white",
)
fig.show()


Technical teams:

- Temperature metrics collapse into two clusters (historical vs. contemporary); anomalies are orthogonal—use them for feature selection.

- Rain anomaly is wholly independent of historical normals—prime predictor for mismatch models.

Farmer:

- You only need to watch one rainfall measure (e.g. monthly) instead of juggling annual, monthly, and contemporary—simplify your weather alerts.

Environmental Agency:

- Focus on anomaly indices for early warning; reduce redundant sensor deployments for historical baselines.

## 3. Weather vs. Disease: The Mismatch Effect

Goal: Visualize how temperature anomalies correlate with disease incidence for both Agricultural and Wild systems.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go


# Your helper
def fit_quadratic_and_ci(df_sub, xcol, ycol, x_min=None, x_max=None, npoints=200):
    X = pd.DataFrame(
        {"const": 1.0, "x1": df_sub[xcol].values, "x2": df_sub[xcol].values ** 2}
    )
    y = df_sub[ycol].values
    model = sm.OLS(y, X).fit()
    if x_min is None:
        x_min = df_sub[xcol].min()
    if x_max is None:
        x_max = df_sub[xcol].max()
    x_pred = np.linspace(x_min, x_max, npoints)
    Xp = pd.DataFrame({"const": 1.0, "x1": x_pred, "x2": x_pred**2})
    pf = model.get_prediction(Xp).summary_frame(alpha=0.05)
    return x_pred, pf["mean"], pf["mean_ci_lower"], pf["mean_ci_upper"]


# Split data
wild_df = df[df["system_type"] == "Natural"]
ag_df = df[df["system_type"] == "Ag"]

# === Panel (a): Wild ===
x_pred, yhat, lo, hi = fit_quadratic_and_ci(wild_df, "temp_anomaly_C", "incidence")

fig_a = go.Figure(
    [
        # raw data
        go.Scatter(
            x=wild_df["temp_anomaly_C"],
            y=wild_df["incidence"],
            mode="markers",
            marker=dict(color=WILD_COLOR, opacity=0.3, size=6),
            name="Wild data",
        ),
        # upper CI line (invisible)
        go.Scatter(x=x_pred, y=hi, mode="lines", line=dict(width=0), showlegend=False),
        # lower CI + fill
        go.Scatter(
            x=x_pred,
            y=lo,
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(200,200,200,0.5)",
            line=dict(width=0),
            name="95% CI",
        ),
        # quadratic fit
        go.Scatter(
            x=x_pred,
            y=yhat,
            mode="lines",
            line=dict(color="black", width=2),
            name="Quadratic fit",
        ),
    ]
)

# vertical line at x=0
fig_a.update_layout(
    shapes=[
        dict(
            type="line",
            x0=0,
            x1=0,
            y0=0,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="gray", dash="dash"),
        )
    ],
    title="(a) Wild: Incidence vs Temperature Anomaly",
    xaxis_title="Temperature Anomaly (°C)",
    yaxis_title="Incidence",
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=50, b=50),
)

fig_a.show()


# === Panel (d): Ag ===
x_pred, yhat, lo, hi = fit_quadratic_and_ci(ag_df, "temp_anomaly_C", "incidence")

fig_d = go.Figure(
    [
        go.Scatter(
            x=ag_df["temp_anomaly_C"],
            y=ag_df["incidence"],
            mode="markers",
            marker=dict(color=AG_COLOR, opacity=0.3, size=6),
            name="Ag data",
        ),
        go.Scatter(x=x_pred, y=hi, mode="lines", line=dict(width=0), showlegend=False),
        go.Scatter(
            x=x_pred,
            y=lo,
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(200,200,200,0.5)",
            line=dict(width=0),
            name="95% CI",
        ),
        go.Scatter(
            x=x_pred,
            y=yhat,
            mode="lines",
            line=dict(color="black", width=2),
            name="Quadratic fit",
        ),
    ]
)

fig_d.update_layout(
    shapes=[
        dict(
            type="line",
            x0=0,
            x1=0,
            y0=0,
            y1=1,
            xref="x",
            yref="paper",
            line=dict(color="gray", dash="dash"),
        )
    ],
    title="(d) Ag: Incidence vs Temperature Anomaly",
    xaxis_title="Temperature Anomaly (°C)",
    yaxis_title="Incidence",
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=50, b=50),
)

fig_d.show()


Technical team:

- Wild systems show a U‐shaped response (negative quadratic), confirming the significant negative anomaly × historical interaction.

- Agricultural curve is near‐linear, validating the buffering effect.

Farmer:

- When your field’s temperature strays ±2 °C from the norm, watch for spikes in wild‐host disease—but crops are safer until extremes.

Environmental Agency:

- Use anomaly thresholds to trigger alerts in protected areas with wild flora—you’ll catch outbreaks at their onset.

Goal: Visualize how rainfall anomalies correlate with disease incidence for both Agricultural and Wild systems.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objects as go


# Your helper
def fit_quadratic_and_ci(df_sub, xcol, ycol, x_min=None, x_max=None, npoints=200):
    X = pd.DataFrame(
        {"const": 1.0, "x1": df_sub[xcol].values, "x2": df_sub[xcol].values ** 2}
    )
    y = df_sub[ycol].values
    model = sm.OLS(y, X).fit()
    if x_min is None:
        x_min = df_sub[xcol].min()
    if x_max is None:
        x_max = df_sub[xcol].max()
    x_pred = np.linspace(x_min, x_max, npoints)
    Xp = pd.DataFrame({"const": 1.0, "x1": x_pred, "x2": x_pred**2})
    pf = model.get_prediction(Xp).summary_frame(alpha=0.05)
    return x_pred, pf["mean"], pf["mean_ci_lower"], pf["mean_ci_upper"]


# Split data
wild_df = df[df["system_type"] == "Natural"]
ag_df = df[df["system_type"] == "Ag"]

# === Panel (a): Wild ===
x_pred, yhat, lo, hi = fit_quadratic_and_ci(wild_df, "rain_anomaly_mm_day", "incidence")

fig_wrain = go.Figure(
    [
        # Scatter
        go.Scatter(
            x=wild_df["rain_anomaly_mm_day"],
            y=wild_df["incidence"],
            mode="markers",
            marker=dict(color=WILD_COLOR, opacity=0.5, size=6),
            name="Wild data",
        ),
        # CI upper (invisible line)
        go.Scatter(x=x_pred, y=hi, mode="lines", line=dict(width=0), showlegend=False),
        # CI lower + fill
        go.Scatter(
            x=x_pred,
            y=lo,
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(200,200,200,0.5)",
            line=dict(width=0),
            name="95% CI",
        ),
        # Fit line
        go.Scatter(
            x=x_pred,
            y=yhat,
            mode="lines",
            line=dict(color="black", width=2),
            name="Quadratic fit",
        ),
    ]
)

fig_wrain.update_layout(
    title="Wild: Incidence vs Rainfall Anomaly",
    xaxis_title="Rainfall Anomaly (mm/day)",
    yaxis_title="Incidence",
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=50, b=50),
)

fig_wrain.show()

# === Panel (d): Ag ===
x_pred, yhat, lo, hi = fit_quadratic_and_ci(ag_df, "rain_anomaly_mm_day", "incidence")

fig_agrain = go.Figure(
    [
        go.Scatter(
            x=ag_df["rain_anomaly_mm_day"],
            y=ag_df["incidence"],
            mode="markers",
            marker=dict(color=AG_COLOR, opacity=0.5, size=6),
            name="Ag data",
        ),
        go.Scatter(x=x_pred, y=hi, mode="lines", line=dict(width=0), showlegend=False),
        go.Scatter(
            x=x_pred,
            y=lo,
            mode="lines",
            fill="tonexty",
            fillcolor="rgba(200,200,200,0.5)",
            line=dict(width=0),
            name="95% CI",
        ),
        go.Scatter(
            x=x_pred,
            y=yhat,
            mode="lines",
            line=dict(color="black", width=2),
            name="Quadratic fit",
        ),
    ]
)

fig_agrain.update_layout(
    title="Ag: Incidence vs Rainfall Anomaly",
    xaxis_title="Rainfall Anomaly (mm/day)",
    yaxis_title="Incidence",
    width=600,
    height=500,
    margin=dict(l=60, r=20, t=50, b=50),
)

fig_agrain.show()


## 4. Pathogen & Host Distributions

Goal: Show the distribution of pathogens and host orders in Agricultural vs. Wild systems.

In [58]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 1) Prepare counts
agent_counts = (
    df.groupby(["Antagonist_type_general", "system_type"], observed=True)
    .size()
    .unstack(fill_value=0)
    .loc[["Virus", "Eukaryotic parasite", "Pest", "Bacteria"]]
)

host_counts = (
    df[df["Host_order"] != "Unknown"]
    .groupby(["Host_order", "system_type"])
    .size()
    .unstack(fill_value=0)
)
# Sort host orders by total desc
host_counts["total"] = host_counts.sum(axis=1)
host_counts = host_counts.sort_values("total", ascending=False).drop(columns="total")

# 2) Create a 1×2 subplot
fig = make_subplots(
    rows=1,
    cols=2,
    subplot_titles=(
        "a) Disease‐causing Agents by System",
        "b) Host Plant Orders by System",
    ),
    column_widths=[0.3, 0.7],
)

# --- Left panel: agents ---
for sys, color in zip(["Natural", "Ag"], [WILD_COLOR, AG_COLOR]):
    fig.add_trace(
        go.Bar(x=agent_counts.index, y=agent_counts[sys], name=sys, marker_color=color),
        row=1,
        col=1,
    )

fig.update_xaxes(title_text="Disease‐causing Agent", row=1, col=1)
fig.update_yaxes(title_text="Observations", row=1, col=1)

# --- Right panel: host orders ---
for sys, color, showleg in zip(["Natural", "Ag"], [WILD_COLOR, AG_COLOR], [True, True]):
    fig.add_trace(
        go.Bar(
            x=host_counts.index,
            y=host_counts[sys],
            name=sys,
            marker_color=color,
            showlegend=showleg,  # legend only once
        ),
        row=1,
        col=2,
    )

fig.update_xaxes(title_text="Host plant order", row=1, col=2, tickangle=45)
fig.update_yaxes(title_text="Observations", row=1, col=2)

# 3) Final layout tweaks
fig.update_layout(
    barmode="stack",
    template="simple_white",
    height=600,
    width=1000,
    legend=dict(x=0.75, y=0.95),
)

# 4) Render inline
import plotly.io as pio

pio.renderers.default = "notebook_connected"
fig.show()

Data Scientist:

- Viruses and eukaryotic parasites dominate agricultural surveys; bacterial and pest data are sparse—balance your modeling samples.

Farmer:

- In your crops, prioritize virus and fungal parasite diagnostics—over 80 % of surveys report these.

Environmental Agency:

- Wild surveys include more “Pinales” and “Vitales” hosts—tailor your wild‐plant protection to these orders.

## 5. Who Lives in What Climate? (Climate Distributions by Pathogen & System)

Goal: Compare the climatic distributions for each pathogen group in Agricultural vs. Wild settings.

**Temperature**

In [84]:
import plotly.express as px

fig = px.violin(
    df,
    y="temp_anomaly_C",
    x="Antagonist_type_general",
    color="system_type",
    box=True,
    points="all",
    hover_data=["Parasite_or_pest", "Host_order"],
    title="Climate (Temperature) Distributions by Pathogen Type & System",
)

fig.update_layout(
    yaxis_title="Temperature anomaly (°C)", xaxis_title="Pathogen Type", template="plotly"
)

fig.show()


In [83]:
import plotly.express as px

fig = px.violin(
    df,
    y="rain_anomaly_mm_day",
    x="Antagonist_type_general",
    color="system_type",
    box=True,
    points="all",
    hover_data=["Parasite_or_pest", "Host_order"],
    title="Climate (Precipitation) Distributions by Pathogen Type & System",
)

fig.update_layout(
    yaxis_title="Rainfall anomaly (mm/day)", xaxis_title="Pathogen Type", template="plotly"
)

fig.show()

- Data Scientist:

Violin shapes show that pests occupy cooler and wetter niches in wild systems, while viruses span warmer, drier extremes in crops—cluster your model residuals by group.

- Farmer:

If you grow legumes (Fabales) under ~20–25 °C, expect more virus incidence; plan your planting window accordingly.

- Environmental Agency:

Wild pathogens see rainfall anomalies up to –4 mm/day; focus surveillance in regions forecast for deep drought.

In [ ]:
import plotly.express as px
import pandas as pd

# 1) Prepare metrics & labels
temp_metrics = [
    "annual_mean_temp_C",
    "monthly_temp_C",
    "contemp_temp_C",
    "temp_anomaly_C",
]
label_map = {
    "annual_mean_temp_C": "Historical annual average temperature (°C)",
    "monthly_temp_C": "Historical monthly average temperature (°C)",
    "contemp_temp_C": "Contemporary average temperature (°C)",
    "temp_anomaly_C": "Temperature anomaly (°C)",
}
# 2) Melt for both systems
melt = df.melt(
    id_vars=["System", "Antagonist_type_general"],
    value_vars=temp_metrics,
    var_name="Metric",
    value_name="Value",
)
melt["Metric Label"] = melt["Metric"].map(label_map)
melt["Metric Label"] = pd.Categorical(
    melt["Metric Label"], categories=[label_map[m] for m in temp_metrics], ordered=True
)

# 3) Draw faceted histogram
fig = px.histogram(
    melt,
    x="Value",
    color="Antagonist_type_general",
    facet_col="System",
    facet_row="Metric Label",
    barmode="stack",
    opacity=0.6,
    category_orders={
        "System": ["Agricultural", "Wild"],
        "Metric Label": [label_map[m] for m in temp_metrics],
    },
    labels={"Value": "Temperature (°C)", "Antagonist_type_general": "Pathogen"},
    title="Temperature Distributions by Pathogen & System",
)
fig.for_each_annotation(
    lambda a: a.update(text=a.text.split("=")[-1].replace("_", " "))
)
fig.update_layout(height=1200, width=800, template="plotly_white")
fig.show()


c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\.venv\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



To keep them separate

In [ ]:
sub_ag = melt[melt["System"] == "Agricultural"]
fig_ag = px.histogram(
    sub_ag,
    x="Value",
    color="Antagonist_type_general",
    facet_row="Metric Label",
    barmode="stack",
    opacity=0.6,
    category_orders={"Metric Label": [label_map[m] for m in temp_metrics]},
    labels={"Value": "Temperature (°C)", "Antagonist_type_general": "Pathogen"},
    title="Ag Systems: Temperature Distributions by Pathogen",
)
fig_ag.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_ag.update_layout(height=1200, width=600, template="plotly_white")
fig_ag.show()


c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\.venv\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [ ]:
sub_wild = melt[melt["System"] == "Wild"]
fig_wd = px.histogram(
    sub_wild,
    x="Value",
    color="Antagonist_type_general",
    facet_row="Metric Label",
    barmode="stack",
    opacity=0.6,
    category_orders={"Metric Label": [label_map[m] for m in temp_metrics]},
    labels={"Value": "Temperature (°C)", "Antagonist_type_general": "Pathogen"},
    title="Wild Systems: Temperature Distributions by Pathogen",
)
fig_wd.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig_wd.update_layout(height=1200, width=600, template="plotly_white")
fig_wd.show()


c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\.venv\Lib\site-packages\plotly\express\_core.py:2044: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



**Rainfall**

- Technical Team:

Horizontal orientation makes heavy‐tailed distributions (e.g. anomalies) easier to read across bins.

Dropdown lets you spot skewness and outliers per metric interactively.

- Farmer:

Click “Temperature anomaly” to see how often your region has >2 °C deviations—plan irrigation or shade interventions.

- Environmental Agency:

Compare “Rainfall anomaly” vs. “Contemporary precipitation” distributions to decide which meteorological service to fund.

## 6. Model Feature Importances

Goal: Visualize the top predictors of disease risk based on your ML model.

In [ ]:
import plotly.express as px

# Suppose you have a DataFrame importance_df with columns ['feature','importance']
importance_df = pd.read_csv("data/processed/feature_importances.csv")

fig = px.bar(
    importance_df.sort_values("importance", ascending=False).head(10),
    x="importance",
    y="feature",
    orientation="h",
    text="importance",
    labels={"importance": "Importance", "feature": "Feature"},
    title="Top 10 Feature Importances",
)
fig.update_traces(marker_color="#636efa")
fig.update_layout(yaxis={"categoryorder": "total ascending"}, width=700, height=400)
fig.show()


- Technical team:

Temperature anomaly leads, followed by precipitation anomaly—confirms our OLS & RandomForest results.

Drop low‐importance covariates to simplify future models.

- Farmer:

Keep an eye on forecasts for both heatwaves and drought spells—you’ll get the highest ROI on prevention.

- Environmental Agency:

Invest in anomaly‐focused monitoring programs (e.g., satellite soil moisture, SPI) rather than static climatologies.



## 7. What’s Next?

For future enhancements:

- Time-slider animation: Watch how survey locations and temperatures change year-by-year.

- Seasonal Trends: Line charts of average incidence by calendar month.

- Sankey Flows: Visualize how incidence zones flow between Agricultural and Wild systems.

- Parallel Coordinates: Explore each survey’s multivariate fingerprint across climate, duration, and incidence.